In [1]:
import pandas as pd
import numpy as np
from us_state_abbrev import us_state_abbrev
import glob

In [2]:
path = r"./data/incomedata" 
files = glob.glob(path + "/*.csv")
years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
li=[]

for (f, y) in zip(files, years):
    df = pd.read_csv(f, index_col = None, header = 0)
    df = df.loc[1:] # removes unnecessary column description
    df['Year'] = y
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
framec = frame.copy()

ValueError: No objects to concatenate

In [10]:
income = framec.loc[:, ['Year', 'NAME' ,'DP03_0051E' ,'DP03_0062E']]
income.rename(columns = {'NAME':'County','DP03_0051E':'Total_Households', 'DP03_0062E':'Med_income'}, inplace = True)

### fixing 'Location' format

In [11]:
income['State'] = income['County'].str.split(',', 1, expand=True)[1]
income['State'] = income['State'].apply(lambda x: x.strip())
income['State'] = income['State'].map(us_state_abbrev)
income['County']=income['County'].str.split(',', 1, expand=True)[0]

 ### changing 'income' to integers

In [12]:
income['Med_income']=income['Med_income'].apply(lambda x: int(x))
income['Total_Households'] = income['Total_Households'].apply(lambda x: int(x))
income['Year'] = pd.to_datetime(income.Year, format='%Y').dt.year

### imputing missing values

In [13]:
income = pd.concat([income, pd.DataFrame([[2015, 'Bayamón Municipio', (71307 + 70124) / 2, (25271 + 24128) / 2, 'PR'], 
                                         [2015, 'Mayagüez Municipio', (29553 + 29138) / 2, (14108 + 14532) / 2, 'PR']], 
                                        columns = ['Year', 'County', 'Total_Households', 'Med_income', 'State'])])

### interpolating for month value

In [14]:
m_income = pd.DataFrame()
for (_, _), df in income.groupby(['County', 'State']):
    df['iChange'] = df['Med_income'].diff() / df['Med_income'].shift()
    df['hChange'] = df['Total_Households'].diff() / df['Total_Households'].shift()
    df = df.iloc[np.arange(df.shape[0]).repeat(12)].reset_index().drop('index', axis = 1)
    df = pd.concat([df, pd.DataFrame(list(range(1, 13)) * int(df.shape[0] / 12), columns = ['Month'])], axis = 1)
    for year in pd.unique(df['Year'])[1:]:
        df.loc[df['Year'] == year, 'Med_income'] = np.array(df.loc[df['Year'] == (year - 1), 'Med_income']) * \
                                                   np.array((((1 + df.loc[df['Year'] == year, 'iChange']) ** (1/12)) ** \
                                                               df.loc[df['Year'] == year, 'Month']))
        df.loc[df['Year'] == year, 'Total_Households'] = np.array(df.loc[df['Year'] == (year - 1), 'Total_Households']) * \
                                                   np.array((((1 + df.loc[df['Year'] == year, 'hChange']) ** (1/12)) ** \
                                                               df.loc[df['Year'] == year, 'Month']))
    df = df[df['Year'] != 2012]
    df = df[['County', 'State', 'Year', 'Month', 'Total_Households', 'Med_income']]
    m_income = pd.concat([m_income, df], axis = 0)

In [17]:
m_income.to_csv('./data/income_and_households.csv')